In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from tensorflow.keras.optimizers import Adam

# 讀取CSV文件
df = pd.read_csv('/content/drive/MyDrive/archive/dogs.csv')

# 選擇指定類別
selected_breeds = [
    "Airedale", "Beagle", "Bloodhound", "Bluetick", "Chihuahua",
    "Collie", "Dingo", "French Bulldog", "German Sheperd",
    "Malinois", "Newfoundland", "Pekinese", "Pomeranian", "Pug", "Vizsla"
]

# 過濾指定的品種
df_selected = df[df['labels'].isin(selected_breeds)]

# 加載和預處理圖像
def load_images(df):
    image_list = []
    for idx, row in df.iterrows():
        img = Image.open('/content/drive/MyDrive/archive/' + row['filepaths'])
        img = img.resize((224, 224))  # 假設使用224x224大小的圖像
        img_array = np.array(img) / 255.0  # 圖像歸一化
        image_list.append(img_array)
    return np.array(image_list)

images = load_images(df_selected)

# 標籤編碼
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df_selected['labels'])

# 分割數據集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 加载预训练的VGG16模型
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # 冻结卷积基

# 建立新模型
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(selected_breeds), activation='softmax')  # 注意这里的输出层仍然使用softmax
])

# 編譯模型，使用SparseCategoricalCrossentropy作为损失函数
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 数据增强
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

# 訓練模型，使用数据增强
model.fit(datagen.flow(X_train, y_train, batch_size=32),
          epochs=10,
          validation_data=(X_test, y_test))

# 評估模型
val_loss, val_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {val_acc}")
# accuracy = 0.712

58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/10
51/51 [==============================] - 1140s 22s/step - loss: 3.9781 - accuracy: 0.1459 - val_loss: 2.0933 - val_accuracy: 0.4144
Epoch 2/10
51/51 [==============================] - 1127s 22s/step - loss: 2.2896 - accuracy: 0.2415 - val_loss: 1.8136 - val_accuracy: 0.4268
Epoch 3/10
51/51 [==============================] - 1126s 22s/step - loss: 2.0337 - accuracy: 0.3402 - val_loss: 1.5301 - val_accuracy: 0.5360
Epoch 4/10
51/51 [==============================] - 1144s 23s/step - loss: 1.8648 - accuracy: 0.3700 - val_loss: 1.2952 - val_accuracy: 0.5955
Epoch 5/10
51/51 [==============================] - 1118s 22s/step - loss: 1.7154 - accuracy: 0.4084 - val_loss: 1.1599 - val_accuracy: 0.6253
Epoch 6/10
51/51 [==============================] - 1116s 22s/step - loss: 1.6438 - accuracy: 0.4327 - val_loss: 1.1098 - val_accuracy: 0.6452
Epoch 7/10
51/51 [==============================] - 1116s 22s/step - loss: 1.